# Generics, Trait, Lifetime

In [9]:
{
  let number_list = vec![34, 50, 25, 100, 65];
  let mut largest = number_list[0];

  for number in number_list {
    if number > largest {
      largest = number;
    }
  }

  println!("The largest number is {}", largest);
}

The largest number is 100


()

We should extract as function when we use sevral times

In [2]:
fn largest(list: &[i32]) -> i32 {
  let mut largest = list[0];

  for &item in list.iter() {
    if item > largest {
      largest = item;
    }
  }
  largest
}

let number_list = vec![34, 50, 25, 100, 65];
let result = largest(&number_list);
println!("The largest number is {}", result);

let number_list = vec![102, 34, 6000, 89, 54, 2, 43, 8];
let result = largest(&number_list);
println!("The largest number is {}", result);

The largest number is 100
The largest number is 6000


We can treat list with different types by generics

In [3]:
fn largest<T>(list: &[T]) -> T {
  let mut largest = list[0];

  for &item in list.iter() {
    if item > largest {
      largest = item;
    }
  }
  largest
}

Error: binary operation `>` cannot be applied to type `T`

But we can't use `>` for every type. We need to ensure the `T` can receive `>`. To ensure that, we cau use `std::cmp::PartialOrd` trait.

## Generics with struct

In [7]:
#[derive(Debug)]
struct Point<T> {
  x: T,
  y: T,
}

In [8]:
// integer
Point { x: 5, y: 10 }

Point { x: 5, y: 10 }

In [10]:
// floar
Point { x: 1.0, y: 4.0 }

Point { x: 1.0, y: 4.0 }

In [11]:
// we can't indicate different typed set
Point { x: 5, y: 4.0 }

Error: mismatched types

We can set differnet types if we treat multiple generics

In [12]:
#[derive(Debug)]
struct Point<T, U> {
  x: T,
  y: U,
}

In [13]:
Point { x: 4, y: 3.0 }

Point { x: 4, y: 3.0 }

## Generics with enum

Like Option<T>, Result<T, E> 

## Generics with method definitioin

In [14]:
#[derive(Debug)]
struct Point<T> {
  x: T,
  y: T,
}

In [15]:
impl<T> Point<T> {
  fn x(&self) -> &T {
    &self.x
  }
}

In [17]:
let p = Point { x: 5, y: 10 };
p.x

5

`impl<T> Point<T>` represents `<T>` in `Point<T>` is generic type. We can concrete type in `impl Point<...>`. 

In [18]:
impl Point<f32> {
  fn distance_from_origin(&self) -> f32 {
    (self.x.powi(2) + self.y.powi(2)).sqrt()
  }
}

In [20]:
let p = Point { x: 2.4, y: -3.5 };
p.distance_from_origin()

4.243819

In [21]:
let p = Point { x: 5, y: 10 };
p.distance_from_origin()

Error: no method named `distance_from_origin` found for struct `Point<{integer}>` in the current scope

In [28]:
#[derive(Debug)]
struct Point2<T, U> {
  x: T,
  y: U,
}

In [ ]:
impl<T, U> Point2<T, U> {
  fn mixup<V, W>(self, other: Point2<V, W>) -> Point2<T, W> {
    Point2 {
      x: self.x,
      y: other.y,
    }
  }
}

In [30]:
let p1 = Point2 { x: 5, y: 10.4 };
let p2 = Point2 { x: "Hello", y: 'c' };

p1.mixup(p2)

Point2 { x: 5, y: 'c' }

## Performance of code with generics

Rust does not require extra cost to run code with generics because Rust do **monomorphization** on compile. Rust replace generic type with actual type from code.

# Define trait

In [2]:
pub trait Summary {
  fn summarize(&self) -> String;
}

In [4]:
pub struct NewsArticle {
  pub headline: String,
  pub location: String,
  pub author: String,
  pub content: String,
}

impl Summary for NewsArticle {
  fn summarize(&self) -> String {
    format!("{}, by {} ({})", self.headline, self.author, self.location)
  }
}

In [6]:
pub struct Tweet {
  pub username: String,
  pub content: String,
  pub reply: bool,
  pub retweet: bool,
}

impl Summary for Tweet {
  fn summarize(&self) -> String {
    format!("{}: {}", self.username, self.content)
  }
}

In [7]:
let tweet = Tweet {
  username: String::from("horse_ebooks"),
  content: String::from("of course, as you probably already know, people"),
  reply: false,
  retweet: false,
};

println!("1 new tweet: {}", tweet.summarize());

1 new tweet: horse_ebooks: of course, as you probably already know, people


Add default implementation for trait

In [8]:
pub trait Summary {
  fn summarize(&self) -> String {
    String::from("(read more ...)")
  }
}

In [15]:
pub struct NewsArticle2 {
  pub headline: String,
  pub location: String,
  pub author: String,
  pub content: String,
}

impl Summary for NewsArticle2 { }


In [16]:
let article = NewsArticle2 {
  headline: String::from("Penguins win the Stanley Cup Championship!"),
  location: String::from("Pittsburgh, PA, USA"),
  author: String::from("Iceburgh"),
  content: String::from(
      "The Pittsburgh Penguins once again are the best \
        hockey team in the NHL.",
  ),
};

println!("New article available! {}", article.summarize());

New article available! (read more ...)


In [23]:
pub trait Summary2 {
  fn summarize_author(&self) -> String;

  fn summarize(&self) -> String {
    format!("(read more from {} ...)", self.summarize_author())
  }
}

In [26]:
pub struct Tweet2 {
  pub username: String,
  pub content: String,
  pub reply: bool,
  pub retweet: bool,
}

impl Summary2 for Tweet2 {
  fn summarize_author(&self) -> String {
    format!("@{}", self.username)
  }
}

In [27]:
let tweet = Tweet2 {
  username: String::from("horse_ebooks"),
  content: String::from("of course, as you probably already know, people"),
  reply: false,
  retweet: false,
};

println!("1 new tweet: {}", tweet.summarize());

1 new tweet: (read more from @horse_ebooks ...)


## Restrict arguments with trait

In [29]:
pub fn notify(item: &impl Summary2) {
  println!("Breaking news! {}", item.summarize())
}

In [32]:
notify(&tweet);

Breaking news! (read more from @horse_ebooks ...)


In [33]:
notify(&String::from("hello"));

Error: the trait bound `String: Summary2` is not satisfied

We can write with `trait bound` syntax. It is useful when we use trait reference sevral times.

```rs
// not so useful
fn notify<T: Summary2>(item: &T) {
  println!("Breaking news! {}", item.summarize())
}

// useful!
fn notify<T: Summary2>(item1: &T, item2: &T) {
  // ...
}
```

We can require arguments to implement sevral traits.

```rs
pub fn notify(item: &(impl Summary + Display)) {
  // ...
```

We can use trait bound syntax
```rs
pub fn notify<T: Summary + Display>(item: &T) {
  // ...
```

We can write trait bound with `where` keyword

```rs
fn some_function<T, U>(t: &T, u: &U) -> i32
    where T: Display + Clone,
          U: Clone + Debug {
```

## Return type which implements trait

In [34]:
fn returns_summarizable () -> impl Summary {
  Tweet {
    username: String::from("horse_ebooks"),
    content: String::from(
        "of course, as you probably already know, people",
    ),
    reply: false,
    retweet: false,
  }
}

In [37]:
returns_summarizable().summarize()

"horse_ebooks: of course, as you probably already know, people"

Resetted here

In [2]:
pub trait Summary {
  fn summarize_author(&self) -> String;

  fn summarize(&self) -> String {
    format!("(read more from {} ...)", self.summarize_author())
  }
}

In [3]:
pub struct Tweet {
  pub username: String,
  pub content: String,
  pub reply: bool,
  pub retweet: bool,
}

impl Summary for Tweet {
  fn summarize_author(&self) -> String {
    format!("@{}", self.username)
  }
}

In [4]:
pub struct NewsArticle {
  pub headline: String,
  pub location: String,
  pub author: String,
  pub content: String,
}

impl Summary for NewsArticle {
  fn summarize_author(&self) -> String {
    format!(":{}", self.author)
  }
}

In [5]:
fn returns_summarizable (switch: bool) -> Box<dyn Summary> {
  if switch {
    Box::new(NewsArticle {
      headline: String::from("Penguins win the Stanley Cup Championship!"),
      location: String::from("Pittsburgh, PA, USA"),
      author: String::from("Iceburgh"),
      content: String::from( "The Pittsburgh Penguins once again are the best hockey team in the NHL."),
    })
  } else {
    Box::new(Tweet {
      username: String::from("horse_ebooks"),
      content: String::from("of course, as you probably already know, people"),
      reply: false,
      retweet: false,
    })
  }
}

In [20]:
returns_summarizable(true).summarize()

"(read more from :Iceburgh ...)"

In [21]:
returns_summarizable(false).summarize()

"(read more from @horse_ebooks ...)"

## ライフタイムで参照を検証する

In [2]:
{
  let r;

  {
    let x = 5;
    r = &x;
  }

  println!("r: {}", r);
}

Error: `x` does not live long enough

### 関数のジェネリックなライフタイム

In [3]:
fn print() {
  let string1 = String::from("abcd");
  let string2 = "xyz";

  let result = longest(string1.as_str(), string2);
  println!("The longest string is {}", result);
}

Error: cannot find function `longest` in this scope

In [5]:
fn longest(x: &str, y: &str) -> &str {
  if x.len() > y.len() {
    x
  } else {
    y
  }
}

Error: missing lifetime specifier

### ライフタイム注釈記法

In [ ]:
&i32        // a reference
&'a i32     // a reference with an explicit lifetime
&'a mut i32 // a mutable reference with an explicit lifetime

In [7]:
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
  if x.len() > y.len() {
    x
  } else {
    y
  }
}

In [8]:
fn print() {
  let string1 = String::from("abcd");
  let string2 = "xyz";

  let result = longest(string1.as_str(), string2);
  println!("The longest string is {}", result);
}

print()

The longest string is abcd


()

In [11]:
fn print() {
  let string1 = String::from("abcd");

  {
    // let string2 = "xyz";
    let string2 = String::from("xyz");
    let result = longest(string1.as_str(), string2.as_str());
    println!("The longest string is {}", result);
  }
}

print()

The longest string is abcd


()

```rs
fn print() {
  | let string1 = String::from("abcd");
  |
  | {
  |  | let string2 = String::from("xyz");
  |  | | let result = longest(string1.as_str(), string2.as_str());
  |  | | println!("The longest string is {}", result);
  |}
  |
}

print()
```

In [14]:
fn print() {
  let string1 = String::from("abcd");
  let result;

  {
    let string2 = String::from("xyz");
    result = longest(string1.as_str(), string2.as_str());
  }
  println!("The longest string is {}", result);
}

print()

Error: `string2` does not live long enough

### 構造体定義のライフタイム注釈

In [18]:
#[derive(Debug)]
struct ImportantExcerpt<'a> {
  part: &'a str,
}

In [23]:
fn call () {
  let novel = String::from("Call me Ishmael. Some years ago ...");
  let first_sentence = novel.split('.').next().expect("Could not find a '.'");

  let i = ImportantExcerpt { part: first_sentence };
  println!("{:#?}", i);
}

call();

ImportantExcerpt {
    part: "Call me Ishmael",
}


### ライフタイム省略

In [ ]:
fn first_word(s: &str) -> &str {
  let bytes = s.as_bytes();

  for (i, &item) in bytes.iter().enumerate() {
      if item == b' ' {
          return &s[0..i];
      }
  }

  &s[..]
}

### メソッド定義におけるライフタイム注釈

In [24]:
impl<'a> ImportantExcerpt<'a> {
  fn level(&self) -> i32 {
      3
  }
}

In [25]:
impl<'a> ImportantExcerpt<'a> {
  fn announce_and_return_part(&self, announcement: &str) -> &str {
      println!("Attention please: {}", announcement);
      self.part
  }
}

In [27]:
impl<'a> ImportantExcerpt<'a> {
  fn announce_and_return_part2(&self, announcement: &str) -> &str {
      println!("Attention please: {}", announcement);
      announcement
  }
}

Error: lifetime mismatch

In [30]:
impl<'a> ImportantExcerpt<'a> {
  fn announce_and_return_part2<'b>(&self, announcement: &'b str) -> &'b str {
      println!("Attention please: {}", announcement);
      announcement
  }
}

In [31]:
use std::fmt::Display;

fn longest_with_an_announcement<T, 'a>(x: &'a str, y: &'a str, ann: T) -> &'a str
    where T: Display
{
    // アナウンス！
    println!("Announcement! {}", ann);
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

Error: lifetime parameters must be declared prior to type parameters

In [32]:
use std::fmt::Display;

fn longest_with_an_announcement<'a, T>(x: &'a str, y: &'a str, ann: T) -> &'a str
    where T: Display
{
    // アナウンス！
    println!("Announcement! {}", ann);
    if x.len() > y.len() {
        x
    } else {
        y
    }
}